# Day 6: Guard Gallivant

## Prompt 

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation.

## Imports

In [1]:
import pandas as pd
from itertools import product

In [2]:
day = "06"

In [3]:
df = pd.read_csv(f"input_day{day}.txt", delim_whitespace=True,header=None)
df

example_input = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

## Part 1

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

Predict the path of the guard. **How many distinct positions will the guard visit before leaving the mapped area?**

Define functions: 

In [4]:
def parse_map(map_input):
    grid = [list(line) for line in map_input.splitlines()]
    return pd.DataFrame(grid)

# Find the guard and see which way they are going
def find_guard(grid):
    for direction in "^>v<":
        for row in grid.index:
            for col in grid.columns:
                if grid.loc[row, col] == direction:
                    return (row, col), direction
    return None, None

# Define rules
DIRECTIONS = {
    "^": (-1, 0),  # Up
    ">": (0, 1),   # Right
    "v": (1, 0),   # Down
    "<": (0, -1)   # Left
}
RIGHT_TURN = {
    "^": ">", ">": "v", "v": "<", "<": "^"
}

# Make them move
def simulate_guard(grid):
    guard_pos, guard_dir = find_guard(grid)
    if guard_pos is None:
        raise ValueError("Guard not found in the map")
    
    visited_positions = set()
    
    # Starting position is always visited
    visited_positions.add(guard_pos)
    
    # Move it until the guard moves out of bounds
    rows, cols = grid.shape
    while True:
        next_row = guard_pos[0] + DIRECTIONS[guard_dir][0]
        next_col = guard_pos[1] + DIRECTIONS[guard_dir][1]
        
        # Check if out of bounds
        if not (0 <= next_row < rows and 0 <= next_col < cols):
            break
        
        # Check the cell in front
        if grid.loc[next_row, next_col] == "#":
            # Obstacle: turn right
            guard_dir = RIGHT_TURN[guard_dir]
        else:
            guard_pos = (next_row, next_col)
            visited_positions.add(guard_pos)
    
    return visited_positions

Solving the example

In [5]:
grid = parse_map(example_input)
visited_positions = simulate_guard(grid)


print(f"Test map - {len(visited_positions)}")

Test map - 41


Solving the actual problem

In [6]:
file_map = pd.DataFrame([list(row[0]) for row in df.itertuples(index=False)])

visited_positions_file = simulate_guard(file_map)

# Output the result for the file input
print(f"File map - {len(visited_positions_file)}")

File map - 5208


## Part 2

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. **How many different positions could you choose for this obstruction?**

Define additional function/modify existing

In [ ]:
def simulate_guard(grid):
    guard_pos, guard_dir = find_guard(grid)
    if guard_pos is None:
        raise ValueError("Guard not found in the map")
    
    visited_states = set()
    visited_positions = set()
    rows, cols = grid.shape

    while True:
        state = (guard_pos, guard_dir)
        if state in visited_states:
            return visited_positions, True  # Cycle detected!!!
        visited_states.add(state)
        visited_positions.add(guard_pos)

        next_row = guard_pos[0] + DIRECTIONS[guard_dir][0]
        next_col = guard_pos[1] + DIRECTIONS[guard_dir][1]

        if not (0 <= next_row < rows and 0 <= next_col < cols):
            break

        if grid.loc[next_row, next_col] == "#":
            guard_dir = RIGHT_TURN[guard_dir]
        else:
            guard_pos = (next_row, next_col)

    return visited_positions, False

def find_obstruction_positions(grid):
    original_grid = grid.copy()
    guard_start, _ = find_guard(grid)
    if guard_start is None:
        raise ValueError("Guard not found in the map")

    visited_positions, _ = simulate_guard(grid)
    obstruction_positions = set()

    for pos in visited_positions:
        row, col = pos
        if pos == guard_start or grid.loc[row, col] != ".":
            continue
        
        grid.loc[row, col] = "#"
        _, is_loop = simulate_guard(grid)
        if is_loop:
            obstruction_positions.add(pos)
        grid.loc[row, col] = original_grid.loc[row, col]  # Reset the cell

    return obstruction_positions


Test Example

In [8]:
obstruction_positions = find_obstruction_positions(grid)
print(f"Example map - {len(obstruction_positions)}")

Example map - 6


Solve actual pb: 

In [9]:
obstruction_positions_file = find_obstruction_positions(file_map)

print(f"File map - {len(obstruction_positions_file)}")

File map - 1972
